In [1]:
import numpy as np

In [2]:
def cd(line, position):
    parts = line.split()
    if parts[2] == "..":
        position = position[:-1]
    elif parts[2] == "/":
        position = position[:1]
    else: 
        position.append(parts[2])
    return position

def get_ls_lines(lines, i_ls):
    n_lines = 0
    for j in range(i_ls+2, len(lines)):
        if lines[j][0] != "$":
            n_lines += 1
        else:
            break
    return lines[i_ls+1:i_ls+n_lines+2]

def ls(terminal_output):
    dict_temp = {}
    for line in terminal_output:
        parts = line.split()
        if parts[0] == "dir":
            dict_temp[parts[1]] = {}
        else:
            dict_temp[parts[1]] = int(parts[0])
    return dict_temp

def get_dict(files_dict, position):
    d = files_dict
    for pos in position[:-1]:
        d = d[pos]
    return d

def construct_files_dict(terminal_output):
    position = ["/"]
    files_dict = {"/" : None}

    for i, line in enumerate(terminal_output):
        if line[:4] == "$ cd":
            position = cd(line, position)
        elif line[:4] == "$ ls":
            d = get_dict(files_dict, position)
            d[position[-1]] = ls(get_ls_lines(terminal_output, i))

    return files_dict

def get_sizes(files_dict, sizes_list):
    size_sum = 0
    for key, value in files_dict.items():
        if isinstance(value, int):
            size_sum += value
        elif isinstance(value, dict):
            sizes_list = get_sizes(value, sizes_list)
            size_sum += sizes_list[-1]
    sizes_list.append(size_sum)
    return sizes_list

def sum_sizes_small_folders(files_dict):
    sizes_list = get_sizes(files_dict, [])[:-1]
    sizes = np.array(sizes_list)
    return np.sum(sizes[sizes <= 100000])

In [3]:
with open("7_test.txt") as f:
    terminal_output_test = f.readlines()
    
files_dict_test = construct_files_dict(terminal_output_test)

sum_sizes_small_folders(files_dict_test)

95437

In [4]:
with open("7.txt") as f:
    terminal_output = f.readlines()
    
files_dict = construct_files_dict(terminal_output)

sum_sizes_small_folders(files_dict)

1989474

In [5]:
sizes = np.array(get_sizes(files_dict, [])[:-1])
min_size = 30000000 + np.max(sizes) - 70000000
folders_big_enough = sizes[sizes >= min_size]
np.sort(folders_big_enough)[0]

1111607